## Data Preprocessing and Classification on the ILPD (Indian Liver Patient Dataset)

#### Dataset link : https://archive.ics.uci.edu/dataset/225/ilpd+indian+liver+patient+dataset

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Reading the dataset

In [4]:
df=pd.read_csv('dataset.csv',header=None)
df.columns=['Age','Gender','TB','DB','Alkphos','Sgpt','Sgot','TP','ALB','A/G Ratio','Target']
df.head()

,Age,Gender,TB,DB,Alkphos,Sgpt,Sgot,TP,ALB,A/G Ratio,Target
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


##### Target : 1->diagnosed with liver disease , 2->not diagnosed

## Exploratory Data Analysis

In [5]:
df.describe()

,Age,TB,DB,Alkphos,Sgpt,Sgot,TP,ALB,A/G Ratio,Target
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,579.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064,1.286449
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.319592,0.452490
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,1.000000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,1.000000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.930000,1.000000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,2.000000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,2.000000


In [6]:
df.info() #checking all the attributes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        583 non-null    int64  
 1   Gender     583 non-null    object 
 2   TB         583 non-null    float64
 3   DB         583 non-null    float64
 4   Alkphos    583 non-null    int64  
 5   Sgpt       583 non-null    int64  
 6   Sgot       583 non-null    int64  
 7   TP         583 non-null    float64
 8   ALB        583 non-null    float64
 9   A/G Ratio  579 non-null    float64
 10  Target     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


### Checking for Null values : 

In [7]:
df.isnull().sum()

Age          0
Gender       0
TB           0
DB           0
Alkphos      0
Sgpt         0
Sgot         0
TP           0
ALB          0
A/G Ratio    4
Target       0
dtype: int64

#### Clearly the A/G Ratio Column has 4 NULL Values. Hence we need to drop these rows.

In [8]:
df=df.dropna()

In [9]:
df.info()  #re-checking for missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 0 to 582
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        579 non-null    int64  
 1   Gender     579 non-null    object 
 2   TB         579 non-null    float64
 3   DB         579 non-null    float64
 4   Alkphos    579 non-null    int64  
 5   Sgpt       579 non-null    int64  
 6   Sgot       579 non-null    int64  
 7   TP         579 non-null    float64
 8   ALB        579 non-null    float64
 9   A/G Ratio  579 non-null    float64
 10  Target     579 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 54.3+ KB


### Encoding Categorical Variables

In [10]:
df['Gender']=df['Gender'].map({"Female":1,"Male":0})  #encoding categorical variable gender as female:1 male:0
df.head()

,Age,Gender,TB,DB,Alkphos,Sgpt,Sgot,TP,ALB,A/G Ratio,Target
0,65,1,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,0,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,0,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,0,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,0,3.9,2.0,195,27,59,7.3,2.4,0.40,1


## Balancing the data

In [11]:
df['Target'].value_counts()

1    414
2    165
Name: Target, dtype: int64

### The Data is imbalanced. 
##### Now we have 2 options. 
##### 1) Oversample the data using SMOTE
##### 2) Undersample the majority data
#### Since it is a medical dataset, I choose undersampling.

### So now undersampling the data to reduce the number of 1's to 190

In [12]:
ones_indices = df[df.iloc[:, -1] == 1].index[:190]  # Get the first 190 indices for 1's
zeros_indices = df[df.iloc[:, -1] == 2].index       # Get all indices for 2's

balanced_indices = ones_indices.union(zeros_indices)
df_balanced = df.loc[balanced_indices].reset_index(drop=True)

num_ones = df_balanced[df_balanced.iloc[:, -1] == 1].shape[0]
num_twos = df_balanced[df_balanced.iloc[:, -1] == 2].shape[0]

print("Number of 1's:", num_ones)  # Should be 190
print("Number of 2's:", num_twos)  # Should be 165

Number of 1's: 190
Number of 2's: 165


In [13]:
df=df_balanced
len(df)

355

In [14]:
df['Target']=df['Target'].map({1:1,2:0})  #Mapping all 2's to 0's for the classification.

##### Target : 1->diagnosed with liver disease , 0->not diagnosed

### Declaring the inputs and targets

In [15]:
inputs=df.iloc[:,0:-1]
targets=df.iloc[:,-1]

inputs.head()

,Age,Gender,TB,DB,Alkphos,Sgpt,Sgot,TP,ALB,A/G Ratio
0,65,1,0.7,0.1,187,16,18,6.8,3.3,0.90
1,62,0,10.9,5.5,699,64,100,7.5,3.2,0.74
2,62,0,7.3,4.1,490,60,68,7.0,3.3,0.89
3,58,0,1.0,0.4,182,14,20,6.8,3.4,1.00
4,72,0,3.9,2.0,195,27,59,7.3,2.4,0.40


### Scaling the inputs

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

inputs = sc.fit_transform(inputs)

inputs

array([[ 1.22528982,  1.79615993, -0.34518268, ...,  0.42104908,
         0.21345894, -0.21644398],
       [ 1.04470116, -0.5567433 ,  1.59055604, ...,  1.08876253,
         0.0871624 , -0.75424006],
       [ 1.04470116, -0.5567433 ,  0.90735414, ...,  0.61182436,
         0.21345894, -0.25005624],
       ...,
       [ 0.32234652,  1.79615993, -0.28824919, ...,  1.37492544,
         1.0975347 ,  0.11967856],
       [ 0.92430872, -0.5567433 , -0.38313834, ..., -0.43743964,
        -1.9335822 , -1.99789348],
       [-0.40000812, -0.5567433 , -0.28824919, ...,  0.89798726,
         1.60272085,  1.8002913 ]])

### Train test Split the data

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size = 0.2, random_state = 365)
y_train=y_train.to_numpy()  #Converting from pandas series to numpy array.
y_test=y_test.to_numpy()

# Classification

## Artificial Neural Network (ANN)

### Creating the Model

In [18]:
tf.random.set_seed(42)  #setting the weights

In [19]:
output_size = 1
hidden_layer_size = 15

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(output_size, activation='sigmoid') 
])

In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(x_train, y_train,epochs=90)

Train on 284 samples
Epoch 1/90
284/284 [==============================] - 0s 2ms/sample - loss: 0.7003 - accuracy: 0.5070
Epoch 2/90
284/284 [==============================] - 0s 51us/sample - loss: 0.6797 - accuracy: 0.5880
Epoch 3/90
284/284 [==============================] - 0s 48us/sample - loss: 0.6650 - accuracy: 0.6408
Epoch 4/90
284/284 [==============================] - 0s 51us/sample - loss: 0.6521 - accuracy: 0.6655
Epoch 5/90
284/284 [==============================] - 0s 48us/sample - loss: 0.6419 - accuracy: 0.6514
Epoch 6/90
284/284 [==============================] - 0s 51us/sample - loss: 0.6310 - accuracy: 0.6761
Epoch 7/90
284/284 [==============================] - 0s 52us/sample - loss: 0.6215 - accuracy: 0.6761
Epoch 8/90
284/284 [==============================] - 0s 49us/sample - loss: 0.6116 - accuracy: 0.6901
Epoch 9/90
284/284 [==============================] - 0s 48us/sample - loss: 0.6020 - accuracy: 0.6901
Epoch 10/90
284/284 [==============================] 

#### Testing the Model

In [22]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

71/71 [==============================] - 0s 1ms/sample - loss: 0.6317 - accuracy: 0.7606

Test loss: 0.63. Test accuracy: 76.06%


In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[26 10]
 [ 7 28]]


## Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
logreg = LogisticRegression()
logreg.fit(x_train,y_train)
print("Logistic Regression Classifier on scaled test data:")
print("Accuracy:", logreg.score(x_test, y_test))

Logistic Regression Classifier on scaled test data:
Accuracy: 0.7746478873239436


## SVM

In [26]:
from sklearn.svm import SVC

In [27]:
svc_clf = SVC(kernel = 'rbf')
svc_clf.fit(x_train,y_train)
print("SVM Classifier on scaled test data:")
print("Accuracy:", svc_clf.score(x_test, y_test))

SVM Classifier on scaled test data:
Accuracy: 0.7746478873239436


## Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
clf3=RandomForestClassifier()
clf3.fit(x_train,y_train)
print('Accuracy:',accuracy_score(clf3.predict(x_test),y_test))

Accuracy: 0.7323943661971831


#### The accuracy in all the 4 models is about 73-79%.

### We will use the ANN.

### Saving the model.

In [29]:
model.save('model.h5') #Saving the model

In [30]:
import pickle  #saving the StandardScaler

with open('scaler.pkl', 'wb') as f:
    pickle.dump(sc, f)